# PROJECT 1

## PART 1: Replicating Models from Class

1. Import all libraries

In [1]:
import numpy as np
import pandas as pd
from math import e
from scipy.optimize import minimize
from scipy.stats import gamma
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.optimize import minimize
from scipy.special import factorial
import math

### Q1: The Poisson Model

#### Consider the example related to billboard exposures from class. The associated data is in the file billboard.csv. Write code to estimate the parameters of the Poisson model using maximum likelihood estimation (MLE).Report your code, the estimated parameters and the maximum value of the log-likelihood.

1. Load the dataset

In [2]:
df = pd.read_csv('billboard.csv')

In [3]:
df.head()

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20


In [4]:
a = np.array(df[['PEOPLE']]) #define the first array that will go through the PLL function
b = np.array(df[['EXPOSURES']]) #define the second array that will go through PLL
lmbda = 1

def PLL(lmbda,a,b):
    c = 0 #we create a variable that will store the cummulative sum of the iteration below
    for i in range(len(a)): #this will go through through the dataset rows
        c += a[i]*np.log(poisson.pmf(b[i], lmbda)) #we get the sum of PEOPLE[i]*LN(POISSON(EXPOSURES[i],lambda))
    return (-1)*c #we need the non-negative cummulative result

soln = minimize(
    PLL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
final_lmbda = soln.x[0]
c = 0
for i in range(len(a)):
    c += a[i]*np.log(poisson.pmf(b[i],final_lmbda))

print("Lambda:{}".format(final_lmbda))
print("Log likelihood:{}".format(c))

Lambda:4.455999290455263
Log likelihood:[-929.04388273]


In [5]:
soln

      fun: array([929.04388273])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.27373677e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 18
      nit: 8
     njev: 9
   status: 0
  success: True
        x: array([4.45599929])

### Q2: The NBD Model

#### Next, write code (for the same dataset) to estimate the parameters of the NBD model using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. Evaluate the NBD model vis-a-vis the Poisson model; explain which is better and why.

In [6]:
df=pd.read_csv('billboard.csv')
df.head()

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20


In [7]:
def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t)))*NBD(params,k-1,t)


def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*np.log(NBD(params,b[i],t))
    return (-1)*nbd

alpha = 1
n = 1
t = 1
solnbd = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
alpha = solnbd.x[0]
n = solnbd.x[1]
nbd = 0
params = (alpha,n)
for i in range(len(a)):
    nbd += a[i]*np.log(NBD(params,b[i],t))
print("Alpha:{}".format(alpha))
print("n:{}".format(n))
print("Log likelihood:{}".format(nbd))

Alpha:0.21751778150917153
n:0.9692594289057651
Log likelihood:[-649.68882748]


In [8]:
solnbd

      fun: array([649.68882748])
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.59161573e-04,  5.68434186e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 11
     njev: 14
   status: 0
  success: True
        x: array([0.21751778, 0.96925943])

### Q3: The Poisson Regression

#### Now consider the khakichinos.com example from class; The associated data is in the file khakichinos.csv.Estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood.

In [9]:
df=pd.read_csv('khakichinos.csv')
a=np.array(df[['LnInc']])
b=np.array(df[['Sex']])
c=np.array(df[['LnAge']])
d=np.array(df[['HHSize']])
k=np.array(df[['NumberofVisits']])

def PR(params,a,b,c,d,k):
    lmbda0, beta1, beta2, beta3, beta4 = params
    p = 0
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        lmbda= lmbda0*math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])
        fact = float(math.factorial(float(k[i])))
        pt1 = pt1 + k[i]*np.log(lmbda)
        pt2 = pt2 + lmbda
        pt3 = pt3 + np.log(fact)
    p = pt1-pt2-pt3
    return (-1)*p

In [10]:
lmbda0=1
beta1=1
beta2=1
beta3=1
beta4=1
params=lmbda0,beta1,beta2,beta3,beta4
soln = minimize(
    PR,
    args = (a,b,c,d,k),
    x0 = np.array((1,1,1,1,1)),
    bounds=[(0.00001, None), (None,None), (None,None), (None,None), (None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

pt1=0
pt2=0
pt3=0

lmbda0 = soln.x[0]
beta1=soln.x[1]
beta2=soln.x[2]
beta3=soln.x[3]
beta4=soln.x[4]
params = (lmbda0, beta1, beta2, beta3, beta4)
for i in range(len(a)):
    lmbda= lmbda0*math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])
    fact = float(math.factorial(float(k[i])))
    pt1 = pt1 + k[i]*np.log(lmbda)
    pt2 = pt2 + lmbda
    pt3 = pt3 + np.log(fact)
p = pt1-pt2-pt3
print("Lambda0: {}".format(lmbda0))
print("Beta1: {}".format(beta1))
print("Beta2: {}".format(beta2))
print("Beta3: {}".format(beta3))
print("Beta4: {}".format(beta4))
print("Log Likelihood: {}".format(p))

Lambda0: 0.04376650692216292
Beta1: 0.09426526605833353
Beta2: 0.004909204441529336
Beta3: 0.5878466719036146
Beta4: -0.03625361512373587
Log Likelihood: [-6291.49715362]


In [11]:
soln

      fun: array([6291.49715362])
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.94058884,  1.16679075,  0.42509782,  0.27666829, -1.05119398])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 654
      nit: 87
     njev: 109
   status: 0
  success: True
        x: array([ 0.04376651,  0.09426527,  0.0049092 ,  0.58784667, -0.03625362])

### Q4: The NBD Regression

#### Consider the khakichinos.com example again. Estimate all relevant parameters for NBD Regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. Evaluate the NBD regression vis-a-vis the Poisson regression; explain which is better and why.

In [12]:
df_k=pd.read_csv('khakichinos.csv')
df_k.head()

,ID,NumberofVisits,LnInc,Sex,LnAge,HHSize
0,1,0,11.379394,1,3.871201,2
1,2,5,9.769956,1,4.043051,1
2,3,0,11.082143,0,3.332205,2
3,4,0,10.915088,1,3.951244,3
4,5,0,10.915088,1,2.833213,3


DEFINING THE EQUATION FOR AN NBD REGRESSION BELOW

In [13]:
df=pd.read_csv('khakichinos.csv')
a=np.array(df[['LnInc']])
b=np.array(df[['Sex']])
c=np.array(df[['LnAge']])
d=np.array(df[['HHSize']])
k=np.array(df[['NumberofVisits']])

def NBDR(params,a,b,c,d,k):
    alpha, n, beta1, beta2, beta3, beta4 = params
    p = 0#we create a variable that will store the cummulative sum of the iteration below
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        pt1 = np.log(math.gamma(n+int(k[i]))) -(np.log(math.gamma(n)) + np.log(factorial(int(k[i]))))
        #pt1 = float(math.gamma(n+k[i]))/(float(math.gamma(n))*float(math.factorial(int(k[i]))))
        denom = math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])
        pt2 = n*(np.log(alpha) - np.log(alpha+denom))
        pt3 = int(k[i])*(np.log(denom) - np.log(alpha+denom))
        #pt2 = (alpha/(alpha+denom))**n
        #pt3 = (e**(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])/(alpha+float((e**(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])))))**k[i]
        p += pt1 + pt2 + pt3
    return -1*p #we'll need the negative so we can use 


In [14]:
alpha=1
n=1
beta1=1
beta2=1
beta3=1
beta4=1
solnbdr = minimize(
    NBDR,
    args = (a,b,c,d,k),
    x0 = np.array((1,1,1,1,1,1)),
    bounds=[(0.000001,None),(0.000001,None),(None,None),(None,None),(None,None),(None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

alpha = solnbdr.x[0]
n=solnbdr.x[1]
beta1=solnbdr.x[2]
beta2=solnbdr.x[3]
beta3=solnbdr.x[4]
beta4=solnbdr.x[5]

p = 0#we create a variable that will store the cummulative sum of the iteration below
pt1 = 0
pt2 = 0
pt3 = 0
for i in range(len(a)):
    pt1 = np.log(math.gamma(n+int(k[i]))) -(np.log(math.gamma(n)) + np.log(factorial(int(k[i]))))
    denom = math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])
    pt2 = n*(np.log(alpha) - np.log(alpha+denom))
    pt3 = int(k[i])*(np.log(denom) - np.log(alpha+denom))
    p += pt1 + pt2 + pt3
print("Alpha: {}".format(alpha))
print("n: {}".format(n))
print("beta1: {}".format(beta1))
print("Beta2: {}".format(beta2))
print("Beta3: {}".format(beta3))
print("Beta4: {}".format(beta4))
print("Log Likelihood: {}".format(p))

Alpha: 8.257849623838672
n: 0.13874665809458253
beta1: 0.07387126962908636
Beta2: -0.009299441596684287
Beta3: 0.9027283477319072
Beta4: -0.024270175256301173
Log Likelihood: -2888.966133576659


In [15]:
solnbdr

      fun: 2888.966133576659
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00195541, -0.18812898, -0.10418262, -0.01191438, -0.03178684,
       -0.0216005 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 616
      nit: 76
     njev: 88
   status: 0
  success: True
        x: array([ 8.25784962,  0.13874666,  0.07387127, -0.00929944,  0.90272835,
       -0.02427018])

### Q5: For each of the models above, can you provide some managerial takeaways?

## PART 2: Analysis of New Data

### Q1: Read books.csv and generate two new datasets

In [16]:
df=pd.read_csv('books.csv')
df=df.loc[df['domain'].isin(["barnesandnoble.com"])]
df=df.reset_index(drop=True)
df

,userid,education,region,hhsz,age,income,child,race,country,domain,date,product,qty,price
0,13503636,4.0,3.0,4,8.0,7,1,1,1,barnesandnoble.com,20070101,TAO TE CHING: A NEW ENGLISH VERSION PAPERBACK,1,8.95
1,13503636,4.0,3.0,4,8.0,7,1,1,1,barnesandnoble.com,20070101,THE COSMO KAMA SUTRA: 77 MIND-BLOWING SEX POSI...,1,12.56
2,13503636,4.0,3.0,4,8.0,7,1,1,1,barnesandnoble.com,20070101,THE COLLECTED POEMS OF EMILY DICKINSON (BARNES...,1,5.95
3,14559596,NaN,4.0,2,10.0,5,1,1,0,barnesandnoble.com,20070101,"SOUPS, SALADS AND STARTERS HARDCOVER",1,7.18
4,14559596,NaN,4.0,2,10.0,5,1,1,0,barnesandnoble.com,20070101,WILLIAMS-SONOMA BAKING HARDCOVER,1,7.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6652,15541746,NaN,1.0,4,6.0,3,1,1,0,barnesandnoble.com,20071231,2008 LOVELY LATIN LADIES WALL CALENDAR WALL CA...,1,6.97
6653,15541746,NaN,1.0,4,6.0,3,1,1,0,barnesandnoble.com,20071231,2008 BRUCE LEE WALL CALENDAR WALL CALENDAR,1,6.49
6654,15541746,NaN,1.0,4,6.0,3,1,1,0,barnesandnoble.com,20071231,2008 SPIDERMAN WALL CALENDAR WALL CALENDAR,1,6.47
6655,15541746,NaN,1.0,4,6.0,3,1,1,0,barnesandnoble.com,20071231,2008 BLACK CATS SQUARE WALL CALENDAR WALL CALE...,1,6.49


### (a) books01.csv, with the structure of the dataset used in the billboard exposures example (i.e., with only two columns – (i) the number purchases, and (ii) the number of people making the corresponding number of purchases)

In [17]:
books01=df.copy()
books01=df.drop(['education','region', 'hhsz', 'age', 'income', 'child', 'race', 'country', 'date', 'price', 'domain', 'product'],axis=1)
books01

,userid,qty
0,13503636,1
1,13503636,1
2,13503636,1
3,14559596,1
4,14559596,1
...,...,...
6652,15541746,1
6653,15541746,1
6654,15541746,1
6655,15541746,1


In [18]:
books01 = books01.groupby(books01['userid']).sum()
books01['userid']=books01.index

In [19]:
books01=books01.copy()
books01

,qty,userid
userid,,
6365661,1,6365661
6396922,1,6396922
8999933,1,8999933
9573834,2,9573834
9576277,5,9576277
...,...,...
15695968,5,15695968
15696910,2,15696910
15698055,9,15698055


In [20]:
books01.rename(columns = {'userid':'ppl_purchasing'}, inplace = True)
books01=books01.reset_index(drop=True)

In [21]:
books01=books01.groupby(['qty']).count()
books01['qty']=books01.index
books01=books01.reset_index(drop=True)
books01

,ppl_purchasing,qty
0,753,1
1,362,2
2,175,3
3,126,4
4,82,5
5,74,6
6,30,7
7,48,8
8,31,9
9,20,10


In [22]:
books01.to_csv('books01.csv', index=False)

### (b) books02.csv, with the structure of the dataset used in the khakichinos.com example, with a new column containing a count of the number of books purchased from barnesandnoble.com by each customer, while keeping the demographic variables (remember to drop date, product, and price).

In [23]:
books01=pd.read_csv('books01.csv')
books01.head()

,ppl_purchasing,qty
0,753,1
1,362,2
2,175,3
3,126,4
4,82,5


In [24]:
df.head()

,userid,education,region,hhsz,age,income,child,race,country,domain,date,product,qty,price
0,13503636,4.0,3.0,4,8.0,7,1,1,1,barnesandnoble.com,20070101,TAO TE CHING: A NEW ENGLISH VERSION PAPERBACK,1,8.95
1,13503636,4.0,3.0,4,8.0,7,1,1,1,barnesandnoble.com,20070101,THE COSMO KAMA SUTRA: 77 MIND-BLOWING SEX POSI...,1,12.56
2,13503636,4.0,3.0,4,8.0,7,1,1,1,barnesandnoble.com,20070101,THE COLLECTED POEMS OF EMILY DICKINSON (BARNES...,1,5.95
3,14559596,NaN,4.0,2,10.0,5,1,1,0,barnesandnoble.com,20070101,"SOUPS, SALADS AND STARTERS HARDCOVER",1,7.18
4,14559596,NaN,4.0,2,10.0,5,1,1,0,barnesandnoble.com,20070101,WILLIAMS-SONOMA BAKING HARDCOVER,1,7.18


In [25]:
pt_a=df[['userid','qty']]
a=pt_a.groupby('userid').sum()
a['userid']=a.index
a.index.names = ['index']
a.head()

,qty,userid
index,,
6365661,1,6365661
6396922,1,6396922
8999933,1,8999933
9573834,2,9573834
9576277,5,9576277


In [26]:
pt_b=df.drop(['qty','price','product','date'], axis=1)
b=pt_b.groupby('userid').mean()
b['userid']=b.index
b.index.names=['index']
b.head()

,education,region,hhsz,age,income,child,race,country,userid
index,,,,,,,,,
6365661,5.0,1.0,2,11.0,7,0,1,0,6365661
6396922,2.0,2.0,2,8.0,4,0,1,0,6396922
8999933,4.0,3.0,5,10.0,3,1,1,0,8999933
9573834,NaN,4.0,2,10.0,5,1,1,0,9573834
9576277,NaN,1.0,3,8.0,7,1,1,0,9576277


In [27]:
books02=pd.merge(a,b,on='userid')
books02.head()

,qty,userid,education,region,hhsz,age,income,child,race,country
0,1,6365661,5.0,1.0,2,11.0,7,0,1,0
1,1,6396922,2.0,2.0,2,8.0,4,0,1,0
2,1,8999933,4.0,3.0,5,10.0,3,1,1,0
3,2,9573834,NaN,4.0,2,10.0,5,1,1,0
4,5,9576277,NaN,1.0,3,8.0,7,1,1,0


In [28]:
books02.to_csv('books02.csv', index=False)

In [29]:
books02=pd.read_csv('books02.csv')
books02.head()

,qty,userid,education,region,hhsz,age,income,child,race,country
0,1,6365661,5.0,1.0,2,11.0,7,0,1,0
1,1,6396922,2.0,2.0,2,8.0,4,0,1,0
2,1,8999933,4.0,3.0,5,10.0,3,1,1,0
3,2,9573834,NaN,4.0,2,10.0,5,1,1,0
4,5,9576277,NaN,1.0,3,8.0,7,1,1,0


### Print the first and last few records of both new datasets.

### Q2: Develop a Poisson model using books01.csv. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant).

In [30]:
books01

,ppl_purchasing,qty
0,753,1
1,362,2
2,175,3
3,126,4
4,82,5
5,74,6
6,30,7
7,48,8
8,31,9
9,20,10


In [31]:
df=books01

a = np.array(df[['ppl_purchasing']]) #define the first array that will go through the PLL function
b = np.array(df[['qty']]) #define the second array that will go through PLL
lmbda = 1

def LL(lmbda,a,b):
    c = 0 #we create a variable that will store the cummulative sum of the iteration below
    for i in range(len(a)): #this will go through through the dataset rows
        c += a[i]*np.log(poisson.pmf(b[i], lmbda)) #we get the sum of PEOPLE[i]*LN(POISSON(EXPOSURES[i],lambda))
    return (-1)*c #we need the negative cummulative result so that we can maximize ll

sol_pn_q2 = minimize(
    LL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)


print("Lambda: {}".format(sol_pn_q2.x[0]))
print("Log likelihood: {}".format(-1*sol_pn_q2.fun[0]))

Lambda: 3.903973987817964
Log likelihood: -7237.86797890748


In [32]:
sol_pn_q2

      fun: array([7237.86797891])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00063665])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 18
      nit: 8
     njev: 9
   status: 0
  success: True
        x: array([3.90397399])

### Q3: Develop a Poisson model using books02.csv, i.e., by ignoring the independent variables available. Report your code, and confirm that the estimated parameters and the maximum value of the log-likelihood are identical to those obtained with the Poisson model developed using books01.csv.

1. Drop the independent variables

In [33]:
new_books2=books02.copy()
new_books2=new_books2.drop(['region','hhsz','age','income','child','race','country','education'],axis=1)
new_books2

,qty,userid
0,1,6365661
1,1,6396922
2,1,8999933
3,2,9573834
4,5,9576277
...,...,...
1807,5,15695968
1808,2,15696910
1809,9,15698055
1810,2,15698341


2. determine the values going into the poisson function

In [34]:
df=books02

a = np.array(df[['userid']])
b = np.array(df[['qty']])
lmbda = 1

def LL(lmbda,a,b):
    c = 0 #we create a variable that will store the cummulative sum of the iteration below
    for i in range(len(a)): #this will go through through the dataset rows
        c += np.log(poisson.pmf(b[i], lmbda)) #we get the sum of PEOPLE[i]*LN(POISSON(EXPOSURES[i],lambda))
    return (-1)*c #we need the non-negative cummulative result so that we can maximize ll

sol_pm_q3 = minimize(
    LL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)


print("Lambda value for poisson model after optimzation {}".format(sol_pm_q3.x[0]))
print("Log likelihood function value for poisson model after optimzation {}".format(-1*sol_pm_q3.fun[0]))

Lambda value for poisson model after optimzation 3.9039701169611294
Log likelihood function value for poisson model after optimzation -7237.867978910017


In [35]:
sol_pm_q3

      fun: array([7237.86797891])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00218279])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 22
      nit: 8
     njev: 11
   status: 0
  success: True
        x: array([3.90397012])

### Q4: Develop an NBD model using books01.csv. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant).

In [36]:
books01.head()

,ppl_purchasing,qty
0,753,1
1,362,2
2,175,3
3,126,4
4,82,5


In [37]:
books02.head()

,qty,userid,education,region,hhsz,age,income,child,race,country
0,1,6365661,5.0,1.0,2,11.0,7,0,1,0
1,1,6396922,2.0,2.0,2,8.0,4,0,1,0
2,1,8999933,4.0,3.0,5,10.0,3,1,1,0
3,2,9573834,NaN,4.0,2,10.0,5,1,1,0
4,5,9576277,NaN,1.0,3,8.0,7,1,1,0


In [38]:
df1=books01.copy()
a = np.array(df1[['ppl_purchasing']])
b = np.array(df1[['qty']])

def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t))) * NBD(params,k-1,t)


def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*np.log(NBD(params,b[i],t))
    return (-1)*nbd

alpha = 1
n = 1
t = 1
sol_nbd4 = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

print("Alpha:{}".format(sol_nbd4.x[0]))
print("n:{}".format(sol_nbd4.x[1]))
print("Log likelihood:{}".format(-1*sol_nbd4.fun[0]))

Alpha:0.3080043615786234
n:1.202447986146062
Log likelihood:-4483.172477500074


In [39]:
sol_nbd4

      fun: array([4483.1724775])
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.03210516,  0.02255547])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 45
      nit: 11
     njev: 15
   status: 0
  success: True
        x: array([0.30800436, 1.20244799])

### Q5: Develop an NBD model using books02.csv (again, ignoring the variables available). Report your code, and confirm that the estimated parameters and the maximum value of the log-likelihood are identical to those obtained with the NBD model developed using books01.csv.

1. Read in the dataset

In [40]:
books02_nbd=books02.copy()
books02_nbd=books02_nbd.drop(['hhsz','region','age','income','child','race','country'],axis=1)
books02_nbd

,qty,userid,education
0,1,6365661,5.0
1,1,6396922,2.0
2,1,8999933,4.0
3,2,9573834,NaN
4,5,9576277,NaN
...,...,...,...
1807,5,15695968,1.0
1808,2,15696910,NaN
1809,9,15698055,NaN
1810,2,15698341,NaN


In [41]:
df2=books02_nbd
a = np.array(df2[['userid']])
b = np.array(df2[['qty']])

def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t))) * NBD(params,k-1,t)


def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += np.log(NBD(params,b[i],t))
    return (-1)*nbd


alpha = 1
n = 1
t = 1
sol_nbdq5 = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

print("Alpha:{}".format(sol_nbdq5.x[0]))
print("n: {}".format(sol_nbdq5.x[1]))
print("Log likelihood:{}".format(-1*sol_nbdq5.fun[0]))

Alpha:0.30799707044210556
n:1.202418887898227
Log likelihood:-4483.172477268105


In [42]:
sol_nbdq5

      fun: array([4483.17247727])
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.01837179,  0.01437002])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 45
      nit: 11
     njev: 15
   status: 0
  success: True
        x: array([0.30799707, 1.20241889])

### Q6: Calculate the values of (i) reach, (ii) average frequency, and (iii) gross ratings points (GRPs) based on the NBD model. Show your work.

In [73]:
alpha = sol_nbdq5.x[0]
n = sol_nbdq5.x[1]

reach (proportion of population who bought a book at least once at barnesandnobles)

In [74]:
reach=100*(1-((alpha/(alpha+t))**n))
reach

82.42851079364824

average frequency (average # of books purchased among those who bought a book)

In [75]:
avg_frequency=n/alpha/(1-(alpha/(alpha+t)))
avg_frequency

5.106413449184959

gross ratings points (average # of books purchased per 100 people)

In [76]:
grps=reach*avg_frequency
grps

420.91405611297296

### Q7: Identify all independent variables with missing values. How many values are missing in each? Drop any variable with many missing values (specify how you are defining ‘many’). If the number of missing values are very few (again, specify how you are defining ‘few’), delete the rows involved. For the remaining variables (if any), replace the missing values with the means of the corresponding variables. Report your code.

In [46]:
books02=pd.read_csv('books02.csv')

In [47]:
books02.head()

,qty,userid,education,region,hhsz,age,income,child,race,country
0,1,6365661,5.0,1.0,2,11.0,7,0,1,0
1,1,6396922,2.0,2.0,2,8.0,4,0,1,0
2,1,8999933,4.0,3.0,5,10.0,3,1,1,0
3,2,9573834,NaN,4.0,2,10.0,5,1,1,0
4,5,9576277,NaN,1.0,3,8.0,7,1,1,0


In [48]:
books02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812 entries, 0 to 1811
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   qty        1812 non-null   int64  
 1   userid     1812 non-null   int64  
 2   education  475 non-null    float64
 3   region     1810 non-null   float64
 4   hhsz       1812 non-null   int64  
 5   age        1811 non-null   float64
 6   income     1812 non-null   int64  
 7   child      1812 non-null   int64  
 8   race       1812 non-null   int64  
 9   country    1812 non-null   int64  
dtypes: float64(3), int64(7)
memory usage: 141.7 KB


In [49]:
books02.education.value_counts()

2.0    162
1.0    123
4.0    123
5.0     64
3.0      3
Name: education, dtype: int64

We can see that education have the most missing values,</br>
we need to delete the column</br>

region and age have 1 & 2 missing values respectively,</br>
so we will drop them

In [50]:
books02=books02.drop(['education'],axis=1)

In [51]:
books02 = books02[~books02['region'].isna()]

In [52]:
books02 = books02[~books02['age'].isna()]

In [53]:
books02.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1809 entries, 0 to 1811
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   qty      1809 non-null   int64  
 1   userid   1809 non-null   int64  
 2   region   1809 non-null   float64
 3   hhsz     1809 non-null   int64  
 4   age      1809 non-null   float64
 5   income   1809 non-null   int64  
 6   child    1809 non-null   int64  
 7   race     1809 non-null   int64  
 8   country  1809 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 141.3 KB


### Q8: Incorporate the available customer characteristics and estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant). What are the managerial takeaways — which customer characteristics seem to be important?

In [54]:
df=books02.copy()
df

,qty,userid,region,hhsz,age,income,child,race,country
0,1,6365661,1.0,2,11.0,7,0,1,0
1,1,6396922,2.0,2,8.0,4,0,1,0
2,1,8999933,3.0,5,10.0,3,1,1,0
3,2,9573834,4.0,2,10.0,5,1,1,0
4,5,9576277,1.0,3,8.0,7,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,5,15695968,2.0,5,10.0,2,1,1,0
1808,2,15696910,3.0,4,8.0,4,1,1,0
1809,9,15698055,3.0,4,4.0,4,1,1,0
1810,2,15698341,4.0,6,8.0,6,1,1,0


In [55]:
a=np.array(df[['region']])
b=np.array(df[['hhsz']])
c=np.array(df[['age']])
d=np.array(df[['income']])
f=np.array(df[['child']])
g=np.array(df[['race']])
h=np.array(df[['country']])
k=np.array(df[['qty']])

def PR2(params,a,b,c,d,f,g,h,k):
    lmbda0, beta1, beta2, beta3, beta4, beta, beta6, beta7 = params
    p = 0
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        lmbda= lmbda0*math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i]+beta5*f[i]+beta6*g[i]+beta7*h[i])
        fact = float(math.factorial(float(k[i])))
        pt1 = pt1 + k[i]*np.log(lmbda)
        pt2 = pt2 + lmbda
        pt3 = pt3 + np.log(fact)
    p = pt1-pt2-pt3
    return (-1)*p

In [56]:
lmbda0=1
beta1=1
beta2=1
beta3=1
beta4=1
beta5=1
beta6=1
beta7=1

params=lmbda0,beta1,beta2,beta3,beta4,beta5,beta6,beta7
sol_pr8 = minimize(
    PR2,
    args = (a,b,c,d,f,g,h,k),
    x0 = np.array((1,1,1,1,1,1,1,1)),
    bounds=[(0.00001, None), (None,None), (None,None), (None,None), (None,None), (None,None), (None,None), (None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)


In [57]:
sol_pr8

      fun: array([7619.77442388])
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00181899, 0.04984031, 0.00409273, 0.04547474, 0.0176442 ,
       0.        , 0.01437002, 0.00100044])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 639
      nit: 60
     njev: 71
   status: 0
  success: True
        x: array([ 3.28862143, -0.00697377, -0.1577432 ,  0.00860118,  0.00521835,
        1.        , -0.13640733, -0.19663273])

### Q9: Estimate all relevant parameters for NBD regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant). What are the managerial takeaways — which customer characteristics seem to be important?

In [58]:
df

,qty,userid,region,hhsz,age,income,child,race,country
0,1,6365661,1.0,2,11.0,7,0,1,0
1,1,6396922,2.0,2,8.0,4,0,1,0
2,1,8999933,3.0,5,10.0,3,1,1,0
3,2,9573834,4.0,2,10.0,5,1,1,0
4,5,9576277,1.0,3,8.0,7,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,5,15695968,2.0,5,10.0,2,1,1,0
1808,2,15696910,3.0,4,8.0,4,1,1,0
1809,9,15698055,3.0,4,4.0,4,1,1,0
1810,2,15698341,4.0,6,8.0,6,1,1,0


In [59]:
df=books02.copy()
df

a=np.array(df[['region']])
b=np.array(df[['hhsz']])
c=np.array(df[['age']])
d=np.array(df[['income']])
f=np.array(df[['child']])
g=np.array(df[['race']])
h=np.array(df[['country']])
k=np.array(df[['qty']])


def NBDR2(params,a,b,c,d,f,g,h,k):
    alpha, n, beta1, beta2, beta3, beta4, beta5, beta6, beta7 = params
    p = 0#we create a variable that will store the cummulative sum of the iteration below
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        pt1 = np.log(math.gamma(n+int(k[i]))) -(np.log(math.gamma(n)) + np.log(factorial(int(k[i]))))
        denom = math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i]+beta5*f[i]+beta6*g[i]+beta7*h[i])
        pt2 = n*(np.log(alpha) - np.log(alpha+denom))
        pt3 = int(k[i])*(np.log(denom) - np.log(alpha+denom))
        p += pt1 + pt2 + pt3
    return -1*p


alpha=1
n=1
beta1=1
beta2=1
beta3=1
beta4=1
beta5=1
beta6=1
beta7=1
sol_nbdr9 = minimize(
    NBDR2,
    args = (a,b,c,d,f,g,h,k),
    x0 = np.array((1,1,1,1,1,1,1,1,1)),
    bounds=[(0.000001,None),(0.000001,None),(None,None),(None,None),(None,None),(None,None),(None,None),(None,None),(None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

In [60]:
sol_nbdr9

      fun: 4468.082925132038
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00154614,  0.00163709,  0.22937456, -0.24356268, -0.32960088,
        0.57934812, -0.04492904,  0.03337846,  0.00745786])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 670
      nit: 54
     njev: 67
   status: 0
  success: True
        x: array([ 0.32624017,  1.21352444, -0.00903476,  0.01035879,  0.01424549,
        0.0170385 ,  0.02143932, -0.12210634, -0.19750576])

In [ ]:
Please refer to our report to managerial 

### Q10: Evaluate all the models developed using the log-likelihood ratio, AIC, and BIC. What are your recommendations on which model to use based on each of these criteria? Are the recommendations consistent? Explain why you are recommending the model you have selected. Are there any significant differences among the results from the models? If so, what exactly are these differences? Discuss what you believe could be causing the differences.

### AIC = 2k - 2*LL, where k = number of parameters

In [61]:
q2k=len(sol_pn_q2.x)
q2ll=(-1)*sol_pn_q2.fun[0]
Q2_aic=2*q2k - 2*q2ll
Q2_aic

14477.73595781496

In [62]:
q3k=len(sol_pm_q3.x)
q3ll=(-1)*sol_pm_q3.fun[0]
Q3_aic=2*q3k - 2*q3ll
Q3_aic

14477.735957820034

In [63]:
q4k=len(sol_nbd4.x)
q4ll=(-1)*sol_nbd4.fun[0]
Q4_aic=2*q4k - 2*q4ll
Q4_aic

8970.344955000148

In [64]:
q5k=len(sol_nbdq5.x)
q5ll=(-1)*sol_nbdq5.fun[0]
Q5_aic=2*q5k - 2*q5ll
Q5_aic

8970.34495453621

In [65]:
q8k=len(sol_pr8.x)
q8ll=(-1)*sol_pr8.fun[0]
Q8_aic=2*q8k - 2*q8ll
Q8_aic

15255.548847764345

In [66]:
q9k=len(sol_nbdr9.x)
q9ll=(-1)*sol_nbdr9.fun
Q9_aic=2*q9k - 2*q9ll
Q9_aic

8954.165850264077

### BIC = k*ln(n)-2*LL, where k=number of parameters and n=number of records

In [67]:
n2=45
q2k=len(sol_pn_q2.x)
q2ll=(-1)*sol_pn_q2.fun[0]
Q2_bic=np.log(n2)*q2k - 2*q2ll
Q2_bic

14479.542620304732

In [68]:
n3=1812
q3k=len(sol_pm_q3.x)
q3ll=(-1)*sol_pm_q3.fun[0]
Q3_bic=np.log(n3)*q3k - 2*q3ll
Q3_bic

14483.238144306637

In [69]:
n4=45
q4k=len(sol_nbd4.x)
q4ll=(-1)*sol_nbd4.fun[0]
Q4_bic=np.log(n4)*q4k - 2*q4ll
Q4_bic

8973.958279979688

In [70]:
n5=1812
q5k=len(sol_nbdq5.x)
q5ll=(-1)*sol_nbdq5.fun[0]
Q5_bic=np.log(n5)*q5k - 2*q5ll
Q5_bic

8981.349327509415

In [71]:
n8=1809
q8k=len(sol_pr8.x)
q8ll=(-1)*sol_pr8.fun[0]
Q8_bic=np.log(n8)*q8k - 2*q8ll
Q8_bic

15299.553083647506

In [72]:
n9=1809
q9k=len(sol_nbdr9.x)
q9ll=(-1)*sol_nbdr9.fun
Q9_bic=np.log(n9)*q9k - 2*q9ll
Q9_bic

9003.670615632635

### Q11: Briefly summarize what you learned from this project. This is an open-ended question, so please include anything you found worthwhile — relating to the modeling process, insights from the process and models, any managerial takeaways that were insightful to you, and so on.

Please refer to our report for answers. 